# **The fifth in-class-exercise (40 points in total, 4/18/2023)**

(20 points) The purpose of the question is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from canvas. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

(7) Word2Vec

(8) BERT

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [2]:
#Write your code here.

import pandas as pd

#Load and format the training data into a data frame
sentiments = []
texts = []

with open('stsa-train.txt', 'r') as f:
    for line in f:
        parts = line.strip().split(' ', 1)
        sentiment = int(parts[0])
        text = parts[1]
        sentiments.append(sentiment)
        texts.append(text)

train_data = pd.DataFrame({'sentiment': sentiments, 'text': texts})

train_data


,sentiment,text
0,1,"a stirring , funny and finally transporting re..."
1,0,apparently reassembled from the cutting-room f...
2,0,they presume their audience wo n't sit still f...
3,1,this is a visually stunning rumination on love...
4,1,jonathan parker 's bartleby should have been t...
...,...,...
6915,1,"painful , horrifying and oppressively tragic ,..."
6916,0,take care is nicely performed by a quintet of ...
6917,0,"the script covers huge , heavy topics in a bla..."
6918,0,a seriously bad film with seriously warped log...


In [4]:
sentiments = []
texts = []

with open('stsa-test.txt', 'r') as f:
    for line in f:
        parts = line.strip().split(' ', 1)
        sentiment = int(parts[0])
        text = parts[1]
        sentiments.append(sentiment)
        texts.append(text)

test_data = pd.DataFrame({'sentiment': sentiments, 'text': texts})
test_data

,sentiment,text
0,0,"no movement , no yuks , not much of anything ."
1,0,"a gob of drivel so sickly sweet , even the eag..."
2,0,"gangs of new york is an unapologetic mess , wh..."
3,0,"we never really feel involved with the story ,..."
4,1,this is one of polanski 's best films .
...,...,...
1816,0,"an often-deadly boring , strange reading of a ..."
1817,0,the problem with concept films is that if the ...
1818,0,"safe conduct , however ambitious and well-inte..."
1819,0,"a film made with as little wit , interest , an..."


In [5]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

import re

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess(sentence):
    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = sentence.replace('{html}',"")
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url = re.sub(r'http\S+', '', cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words = [stemmer.stem(w) for w in filtered_words]
    lemma_words = [lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(lemma_words)

# Apply the preprocessing to the training data
train_data['text'] = train_data['text'].apply(preprocess)

# Apply the preprocessing to the test data
test_data['text'] = test_data['text'].apply(preprocess)

# Convert the text data into a numerical representation using Bag of Words
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_data['text'])
y_train = train_data['sentiment']
X_test = vectorizer.transform(test_data['text'])
y_test = test_data['sentiment']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Supraja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Supraja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
train_data['text']

0       stir funni final transport imagin beauti beast...
1        appar reassembl cut room floor given daytim soap
2       presum audienc sit still sociolog lesson howev...
3       visual stun rumin love memori histori war art ...
4       jonathan parker bartlebi end modern offic anom...
                              ...                        
6915                pain horrifi oppress tragic film miss
6916    take care nice perform quintet actress nonethe...
6917    script cover huge heavi topic bland surfacey w...
6918    serious bad film serious warp logic writer dir...
6919           delici nonsens comedi citi come apart seam
Name: text, Length: 6920, dtype: object

In [8]:
test_data['text']

0                                 movement yuk much anyth
1       gob drivel sickli sweet even eager consum moor...
2       gang new york unapologet mess whose save grace...
3       never realli feel involv stori idea remain abs...
4                                  one polanski best film
                              ...                        
1816    often deadli bore strang read classic whose wi...
1817      problem concept film concept poor one save movi
1818    safe conduct howev ambiti well intent fail hit...
1819    film made littl wit interest profession artist...
1820                               real damn funni either
Name: text, Length: 1821, dtype: object

In [9]:
from sklearn.model_selection import train_test_split

# Split the training data into training and validation data with a ratio of 80:20
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [10]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [11]:
# Create the Multinomial Naive Bayes model and perform 10-fold cross-validation on the training data
nb = MultinomialNB()
scores = cross_val_score(nb, X_train, y_train, cv=10, scoring='accuracy')
print("Multinomial Naive Bayes Mean accuracy: ", scores.mean())
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_val)
print("Multinomial Naive Bayes Classification Report:\n", classification_report(y_val, y_pred_nb))


Multinomial Naive Bayes Mean accuracy:  0.7781823463745504
Multinomial Naive Bayes Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.74      0.77       671
           1       0.77      0.84      0.81       713

    accuracy                           0.79      1384
   macro avg       0.79      0.79      0.79      1384
weighted avg       0.79      0.79      0.79      1384



In [12]:
# Create the SVM model and perform 10-fold cross-validation on the training data
svm = SVC()
scores = cross_val_score(svm, X_train, y_train, cv=10, scoring='accuracy')
print("SVM Mean accuracy: ", scores.mean())
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_val)
print("SVM Classification Report:\n", classification_report(y_val, y_pred_svm))

SVM Mean accuracy:  0.7577623856744635
SVM Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.73      0.77       671
           1       0.77      0.84      0.80       713

    accuracy                           0.79      1384
   macro avg       0.79      0.78      0.78      1384
weighted avg       0.79      0.79      0.78      1384



In [13]:
# Create the KNN model and perform 10-fold cross-validation on the training data
knn = KNeighborsClassifier()
scores = cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy')
print("KNN Mean accuracy: ", scores.mean())
knn.fit(X_train, y_train)
y_pred_knn = knn.predict(X_val)
print("KNN Classification Report:\n", classification_report(y_val, y_pred_knn))

KNN Mean accuracy:  0.5867069022920598
KNN Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.61      0.57       671
           1       0.58      0.51      0.54       713

    accuracy                           0.56      1384
   macro avg       0.56      0.56      0.56      1384
weighted avg       0.56      0.56      0.56      1384



In [14]:
# Create the Decision Tree model and perform 10-fold cross-validation on the training data
dt = DecisionTreeClassifier()
scores = cross_val_score(dt, X_train, y_train, cv=10, scoring='accuracy')
print("Decision Tree Mean accuracy: ", scores.mean())

dt.fit(X_train, y_train)
y_pred = dt.predict(X_val)
print("Classification Report for Decision Tree Model: \n", classification_report(y_val, y_pred))

Decision Tree Mean accuracy:  0.6585891853428298
Classification Report for Decision Tree Model: 
               precision    recall  f1-score   support

           0       0.68      0.67      0.67       671
           1       0.69      0.71      0.70       713

    accuracy                           0.69      1384
   macro avg       0.69      0.69      0.69      1384
weighted avg       0.69      0.69      0.69      1384



In [15]:
# Create the Random Forest model and perform 10-fold cross-validation on the training data
rf = RandomForestClassifier()
scores = cross_val_score(rf, X_train, y_train, cv=10, scoring='accuracy')
print("Random Forest Mean accuracy: ", scores.mean())
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_val)
print("Classification Report for Decision Tree Model: \n", classification_report(y_val, y_pred_rf))

Random Forest Mean accuracy:  0.7268744165399103
Classification Report for Decision Tree Model: 
               precision    recall  f1-score   support

           0       0.77      0.67      0.71       671
           1       0.72      0.81      0.76       713

    accuracy                           0.74      1384
   macro avg       0.74      0.74      0.74      1384
weighted avg       0.74      0.74      0.74      1384



In [16]:
# Create the XGBoost model and perform 10-fold cross-validation on the training data
xgb = XGBClassifier()
scores = cross_val_score(xgb, X_train, y_train, cv=10, scoring='accuracy')
print("XGBoost Mean accuracy: ", scores.mean())
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_val)
print("Classification Report for Decision Tree Model: \n", classification_report(y_val, y_pred_xgb))

XGBoost Mean accuracy:  0.719656484812085
Classification Report for Decision Tree Model: 
               precision    recall  f1-score   support

           0       0.73      0.73      0.73       671
           1       0.74      0.74      0.74       713

    accuracy                           0.73      1384
   macro avg       0.73      0.73      0.73      1384
weighted avg       0.73      0.73      0.73      1384



In [17]:
#Training a Word2Vec Model
nltk.download('punkt')
# Tokenize the reviews into sentences
sentences = [nltk.sent_tokenize(review) for review in train_data['text']]

from gensim.models import Word2Vec

# train model
model = Word2Vec(sentences, min_count=1, vector_size=300)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.key_to_index)
# access vector for one sentence
print(model.wv.get_vector('offer new insight matter charact exactli spring life'))
# save model
model.save('model.bin')
# load model
new_model = Word2Vec.load('model.bin')
print(new_model)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Supraja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Word2Vec<vocab=6899, vector_size=300, alpha=0.025>
[-2.7902445e-04 -3.2790855e-03 -1.0501762e-03  4.1931868e-05
  1.8708174e-03 -2.8985785e-03  1.2176129e-03 -3.1945610e-03
  1.6634866e-03  2.5917320e-03  1.8078133e-03  2.8512545e-03
 -3.2797336e-04 -2.6543073e-03  1.3295221e-03  1.1467402e-03
  2.2770402e-03  2.2057907e-03 -1.3995854e-03  1.6731803e-03
  2.3202591e-03  1.2234731e-03 -3.3294440e-03 -3.0530309e-03
 -2.4362532e-03  1.4558275e-03  2.7001460e-04 -2.3820950e-03
 -8.9830277e-04  2.3827453e-03  1.3769603e-03 -5.2890100e-04
 -1.7734810e-03 -3.1183537e-03 -2.6763459e-03  2.2984720e-03
 -1.7644898e-03 -5.0507742e-04 -1.1627702e-03 -2.7044734e-03
  2.2734403e-04 -1.1609737e-03 -6.9915928e-04  2.2314782e-03
 -2.0104253e-03 -1.7740738e-03  2.6385942e-03  1.4671132e-03
  1.4610529e-04  9.0708176e-04 -9.7704690e-04  3.5650333e-04
 -9.0388855e-04 -1.8372322e-03  2.3970092e-03 -3.0308127e-04
  2.1171998e-03  1.6272315e-03  1.3714135e-03 -1.5134811e-05
 -4.1125735e-04 -5.2097639e-05  1.

In [18]:
import numpy as np
import string
import warnings
warnings.filterwarnings('ignore')

# Define a function to vectorize a sentence
def vectorize_sentence(sentence, model):
    """
    Given a sentence and a Word2Vec model, return the sentence vector
    by averaging the word vectors in the sentence.
    """
    # Combine the words in the sentence into a single string
    sentence = ' '.join(sentence)
    # Remove punctuation and convert to lowercase
    sentence = sentence.translate(str.maketrans('', '', string.punctuation)).lower()
    # Split sentence into words
    words = sentence.split()
    # Filter out words that are not in the Word2Vec model
    words = [word for word in words if word in model.wv]
    # If there are no words in the sentence that are in the Word2Vec model, return a zero vector
    if len(words) == 0:
        return np.zeros(model.vector_size)
    # Otherwise, return the average of the word vectors in the sentence
    else:
        return np.mean([model.wv[word] for word in words], axis=0)


# Vectorize the train set
X_train_vec = np.array([vectorize_sentence(sentence, model) for sentence in sentences])
y_train = train_data['sentiment']

# Split train set into train and validation set
X_train, X_val, y_train, y_val = train_test_split(X_train_vec, y_train, test_size=0.2, random_state=42)

# Vectorize the test set
X_test = np.array([vectorize_sentence(sentence, model) for sentence in test_data['text']])
y_test = test_data['sentiment']

from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Train SVM model
svm = SVC(kernel='linear', random_state=42,)
# Perform 10-fold cross-validation on the training data
scores = cross_val_score(svm, X_train, y_train, cv=10)

# Print the cross-validation scores
print('Mean cross-validation score:', scores.mean())
svm.fit(X_train, y_train)

# Make predictions on the validation set
y_pred = svm.predict(X_val)

# Evaluate the model
print(classification_report(y_val, y_pred))

Mean cross-validation score: 0.5233018455291452
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       671
           1       0.52      1.00      0.68       713

    accuracy                           0.52      1384
   macro avg       0.26      0.50      0.34      1384
weighted avg       0.27      0.52      0.35      1384



(20 points) The purpose of the question is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K-means

DBSCAN

Hierarchical clustering

Word2Vec

BERT

You can refer to of the codes from  the follwing link below. 
https://www.kaggle.com/karthik3890/text-clustering 

In [19]:
#importing necessary libraries
import re
from textblob import TextBlob
from nltk.stem import PorterStemmer
from textblob import Word
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Supraja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [21]:
import pandas as pd
df = pd.read_csv('Amazon_Unlocked_Mobile.csv')


In [22]:
new =df.head(1000)
new['Reviews']

0      I feel so LUCKY to have found this used (phone...
1      nice phone, nice up grade from my pantach revu...
2                                           Very pleased
3      It works good but it goes slow sometimes but i...
4      Great phone to replace my lost phone. The only...
                             ...                        
995    It's a decent for the price.. I've had this on...
996                                   Is good cell phone
997    Amazing phone. Cables and case included, also ...
998                                             Excelent
999       Excellent, it meets the requirements requested
Name: Reviews, Length: 1000, dtype: object

In [25]:
#Implementing kmeans usind TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer  # Import the TfidfVectorizer class for generating TF-IDF features

# Instantiate a TfidfVectorizer object with default parameters
tfidf_vect = TfidfVectorizer()

# Use the fit_transform method of the TfidfVectorizer object to generate a TF-IDF matrix from the 'Lemmatization' column of the 'new' dataframe
tfidf = tfidf_vect.fit_transform(new['Lemmatization'].values)

# Print the shape of the resulting TF-IDF matrix
tfidf.shape

KeyError: 'Lemmatization'

In [26]:
from sklearn.cluster import KMeans  # Import the KMeans class from scikit-learn

# Instantiate a KMeans object with 10 clusters and a random state of 99
model_tf = KMeans(n_clusters=10, random_state=99)

# Use the fit method of the KMeans object to fit the model to the TF-IDF matrix generated in the previous step
model_tf.fit(tfidf)

NameError: name 'tfidf' is not defined

In [27]:
# Assign the cluster labels generated by the KMeans algorithm to a variable
labels_tf = model_tf.labels_

# Assign the cluster centers generated by the KMeans algorithm to a variable
cluster_center_tf = model_tf.cluster_centers_

# Print the cluster centers to the console
print(cluster_center_tf)

AttributeError: 'KMeans' object has no attribute 'labels_'

In [ ]:
# Use the get_feature_names_out method of the TfidfVectorizer object to extract the terms in the corpus
terms1 = tfidf_vect.get_feature_names_out()

# Print the first 100 terms to the console
print(terms1[1:100])

In [ ]:
df1 = new  # Assign the DataFrame 'new' to a new variable 'df1'
df1['Tfidf Clus Label'] = model_tf.labels_  # Add a new column 'Tfidf Clus Label' to 'df1' containing the clustering labels obtained from 'model_tf'
df1[['Lemmatization','Tfidf Clus Label']].head()  # Select the 'Lemmatization' and 'Tfidf Clus Label' columns from 'df1' and display the first five rows using the `head()` method


In [ ]:
df1.groupby(['Tfidf Clus Label'])['Reviews'].count()


In [ ]:
print("Top clusters:")
order_centroids = model_tf.cluster_centers_.argsort()[:, ::-1]
for i in range(10):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :10]:
        print(' %s' % terms1[ind], end='')
        print()

In [ ]:
for i in range(10):
    print("4 reviews of ensured to cluster ", i)
    print("-" * 70)
    print(df1.iloc[df1.groupby(['Tfidf Clus Label']).groups[i][0]]['Reviews'])
    print('\n')
    print(df1.iloc[df1.groupby(['Tfidf Clus Label']).groups[i][2]]['Reviews'])
    print('\n')
    print(df1.iloc[df1.groupby(['Tfidf Clus Label']).groups[i][6]]['Reviews'])
    print('\n')
    print("_" * 70)

In [ ]:
# Generating bag of words features and Implementing K-means.
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
bow = count_vect.fit_transform(new['Reviews'].values)


In [ ]:
from sklearn.cluster import KMeans  # Importing the KMeans clustering algorithm from sklearn.cluster library
model = KMeans(n_clusters = 10,init='k-means++',random_state=99)  # Creating an instance of the KMeans clustering algorithm with 10 clusters, k-means++ initialization, and a fixed random state of 99.
model.fit(bow)  # Fitting the KMeans clustering model on the bag-of-words (bow) matrix.


In [ ]:
labels = model.labels_  # Obtaining the cluster labels for each data point in the input matrix using the 'labels_' attribute of the KMeans model
cluster_center = model.cluster_centers_  # Obtaining the cluster centers (centroids) for each of the clusters using the 'cluster_centers_' attribute of the KMeans model

from sklearn import metrics
print(metrics.silhouette_score(bow, labels, metric='euclidean'))


new['Bow Label'] = model.labels_ 
new[['Lemmatization','Bow Label']].head()

In [ ]:
#Implementing DBSCAN
from sklearn.cluster import DBSCAN  # Importing the DBSCAN clustering algorithm from the sklearn.cluster library
import numpy as np  # Importing the numpy library and aliasing it as np

minPts = 2 * 100  # Setting the value of the minimum points parameter for DBSCAN algorithm to be twice the dimensionality of the dataset

def lower_bound(nums, target):  # Defining a function to return the number in the array just greater than or equal to itself
    l, r = 0, len(nums) - 1
    while l <= r:  # Implementing binary search to find the nearest number
        mid = int(l + (r - l) / 2)
        if nums[mid] >= target:
            r = mid - 1
        else:
            l = mid + 1
    return l

def compute200thnearestneighbour(x, data):  # Defining a function to compute the 200th nearest neighbor of a point in the dataset
    dists = []  # Initializing an empty list to store the distances
    for val in data:  # Computing the distance between the given point and all other points in the dataset
        dist = np.sum((x - val) **2 )
        if (len(dists) == 200 and dists[199] > dist): 
          l = int(lower_bound(dists, dist)) 
          if l < 200 and l >= 0 and dists[l] > dist:
              dists[l] = dist
        else:
          dists.append(dist)
          dists.sort()
    
    return dists[199]  # Returning the distance to the 200th nearest neighbor

In [ ]:
list_of_sent_train = list()

for i in new["Lower casing"].values:
  list_of_sent_train.append(i.split())

In [ ]:
import gensim

# Train Word2Vec model
list_of_sent_train = list()
for i in new["Lower casing"].values:
  list_of_sent_train.append(i.split())
w2v_model = gensim.models.Word2Vec(list_of_sent_train, workers=4)

import numpy as np

# Create sentence vectors using trained Word2Vec model
sent_vectors = []
count = 1
for sent in list_of_sent_train:
    sent_vec = np.zeros(100)
    cnt_words = 1
    for word in sent:
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
sent_vectors = np.array(sent_vectors)
sent_vectors = np.nan_to_num(sent_vectors)

# Calculate 200th nearest neighbor for each sentence vector
twohundrethneigh = []
for val in sent_vectors[:300]:
    twohundrethneigh.append(compute200thnearestneighbour(val, sent_vectors[:300]))
twohundrethneigh.sort()

# Train DBSCAN model
model = DBSCAN(eps=5, min_samples=minPts, n_jobs=-1)
model.fit(sent_vectors)

In [ ]:
new['AVG-W2V Clus Label'] = model.labels_
new[['Lemmatization','AVG-W2V Clus Label']].head()


In [ ]:
#Implementing Hierarchial Clustering
import scipy
from scipy.cluster import hierarchy

from sklearn.cluster import AgglomerativeClustering

# Instantiate AgglomerativeClustering object with the number of clusters, affinity measure, and linkage criteria
cluster = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward') 

# Fit the clustering model to the sent_vectors data using the fit_predict() method
Agg = cluster.fit_predict(sent_vectors)

# Create a new DataFrame called "hier" with the same data as the "new" DataFrame
hier = new

# Assign the cluster labels to the AVG-W2V Clus Label column of the hier DataFrame
hier['AVG-W2V Clus Label'] = cluster.labels_

# Display the first 5 rows of the hier DataFrame
hier.head(5)

In [ ]:
hier.groupby(['AVG-W2V Clus Label'])['Reviews'].count()


In one paragraph, please compare the results of K-means, DBSCAN, Hierarchical clustering, Word2Vec, and BERT.

In [2]:
#You can write you answer here. (No code needed)
Based on Euclidean distance, K-means allocates each data point to the closest cluster center.
Given that the clusters are clearly delineated based on categories or products, K-means outperforms DBSCAN on TFIDF. 
While DBSCAN uses a density criterion to group together data points that are close to one another. 
However, all reviews are evenly grouped for average word vectors. 
DBSCAN performance on a dataset with is subpar since it clusters all reviews together.
By repeatedly merging or dividing existing clusters according to their proximity, hierarchical clustering creates a hierarchy of clusters. 
Because the clusters are not evenly distributed and are challenging to identify, hierarchical clustering does not perform well.

